In [1]:
import xarray as xr
from pylab import *
from datetime import *

In [2]:
from dask.distributed import Client,LocalCluster
client =  Client(scheduler_file='scheduler.json')
client

Client Scheduler: tcp://10.9.1.41:8789 Dashboard: http://10.9.1.41:8820/status,Cluster Workers: 32 Cores: 32 Memory: 67.42 GB


In [3]:
#source: https://github.com/COSIMA/ACCESS-OM2-1-025-010deg-report/blob/master/figures/ice_timeseries/ice_timeseries.ipynb

def fixcicetime(da):
    '''
    Correct the time coordinate in DataArray from CICE netcdf file.
    
    CICE netcdf files unhelpfully have a time coordinate which is just after the end of the averaging period, 
    e.g. the time stamp for a January average is 1 February, which messes up groupby month etc.
    
    This function just subtracts 12 hours to put it in the correct month (and day, for daily means).
    
    PR 109 gives an option to fix this:
    https://github.com/OceansAus/cosima-cookbook/pull/109
    
    '''
    try:
        da['time'] = da.time - timedelta64(12, 'h')
    except:
        da['time'] = da.time - timedelta(hours=12)  # for 01deg which for some reason uses cftime
    return da

# use this for DataSet: replaces the bad time dimension with the average of time_bounds.
#     The time type is also changed to datetime64[ns]
#     ds['time'] = ds.time_bounds.astype('int64').mean(axis=1).astype('datetime64[ns]')

In [4]:
datadir = '/g/data/v45/hh0162/projects/icebgc/output_access-om2/1deg_jra55_ryf_20190729/output'


In [5]:
griddir = datadir+'001/ocean/ocean_grid.nc'
ds_grid = xr.open_dataset(griddir)

z_upper = 300

dsx = xr.open_dataset(datadir+'001/ocean/ocean.nc')
st_ocean = dsx['st_ocean'].sel(st_ocean=slice(0,z_upper))
n_upper = size(st_ocean)
zmask = dsx.temp.mean('time')/dsx.temp.mean('time')
zarea = ds_grid.area_t*zmask

#dsx_ice = xr.open_mfdataset(datadir+'001/ice/OUTPUT/*.nc')
#zarea = dsx_ice.tarea.mean('time')*zmask


/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [6]:
n_yrs = 500 
exp0 = 1
exp1 = 64
l_exp = []
for it in arange(exp0,exp1+1):
    if it < 10:
        l_exp.append('00'+str(it))
    elif it > 9 or it < 100:
        l_exp.append('0'+str(it))
    else:
        l_exp.append(str(it))
print(l_exp)

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064']


In [7]:
var1docean = [
    'total_co2_flux',
    'temp_global_ave',
    'salt_global_ave',
    'temp_surface_ave',
    'salt_surface_ave',
]

uni1docean = [
    'Pg/Yr',
    'deg.C',
    'psu',
    'deg.C',
    'psu'
]

var3docean = [
    'temp',
    'salt',
    'no3',
    'phy',
    'zoo',
    'det',
    'fe',
    'o2',
    'alk',
    'dic',
    'adic',
    'caco3'
]

uni3docean = [
    'deg.C',
    'psu',
    'mmol/m3',
    'mmol-N/m3',
    'mmol-N/m3',
    'mmol-N/m3',
    'mmol/m3',
    'mmol/m3',
    'mmol/m3',
    'mmol/m3',
    'mmol/m3',
    'mmol/m3'
]

var2dice = [
#    'Tair_m',
    'aice_m',
    'hs_m',
    'hi_m',
]

uni2dice = [
#    'deg.C',
    '10$^6$ km$^2$',
    '10$^3$ km$^3$',
    '10$^3$ km$^3$',
]

### northern and southern hemisphere ice quantities

### 20190902: latitude slice does not work because dimension "nj" is not in degree N/S, but is simply indices. not suitable for xarray.

In [8]:
dsx['yt_ocean'] = ds_grid.yt_ocean.values

In [9]:
#calculating
if 1 == 1:
    ts_air = zeros((n_yrs))
    ts_ice = zeros((size(var2dice),2,n_yrs))
    i_yr = 0
    for it in arange(0,5):#size(l_exp)):
        dsx = xr.open_mfdataset(datadir+l_exp[it]+'/ice/OUTPUT/*.nc')
        fixcicetime(dsx)
        dsx = dsx.groupby('time.year').mean('time')
        for it2 in range(size(var2dice)):
            ice_volume = (dsx[var2dice[it2]]*dsx.tarea.values)
            #Northern Hemisphere
            ts_ice[it2,0,i_yr:i_yr+size(ice_volume,0)] = sum(ice_volume.sel(nj=slice(30,90)),axis=(1,2))
            #Southern Hemisphere
            ts_ice[it2,1,i_yr:i_yr+size(ice_volume,0)] = sum(ice_volume.sel(nj=slice(-90,-30)),axis=(1,2))
        #global surface air temperature
        ice_volume = (dsx['Tair_m']*dsx.tarea.values*dsx.tmask.values)
        ts_air[i_yr:i_yr+size(ice_volume,0)] = sum(ice_volume,axis=(1,2))/sum(dsx.tarea.values*dsx.tmask.values)  
        i_yr += size(ice_volume,0)
        save('ts_ice',ts_ice)
        save('ts_air',ts_air)
        print(str(it)+'/'+str(size(l_exp)))
#plotting
else:
    #plot air temp.
    ts_air = load('ts_air.npy')
    figure()
    title(var2dice[0])
    plot(arange(n_yrs),ts_air)
    xlabel('Year')
    ylabel('deg.C')
    savefig('ts_air.pdf',dpi=300)
    #plot ice
    ts_ice = load('ts_ice.npy') * 1e-12
    figure(figsize=(12,8))
    for it in range(size(ts_ice,0)):
        subplot(size(ts_ice,0),1,it+1)
        title(var2dice[it])
        plot(arange(n_yrs),ts_ice[it,0,:],label='NH')
        plot(arange(n_yrs),ts_ice[it,1,:],label='SH')
        xlabel('Year')
        ylabel(uni2dice[it])
        if it == 0:
            legend()
    tight_layout()
    savefig('ts_ice.pdf',dpi=300)
    

/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  import sys
/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


0/64


/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  import sys
/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


1/64


/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  import sys
/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


2/64


/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  import sys
/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


3/64


/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  import sys
/apps/pangeo/2019.10/envs/pangeo/lib/python3.7/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


4/64


### time series of 3d ocean fields in the upper X m

In [10]:
#calculating time series
if 1 == 1:
    ts_bgc = zeros((size(var3docean),n_yrs,n_upper))
    i_yr = 0    
    for it in arange(0,exp1-exp0+1):
        dsx = xr.open_dataset(datadir+l_exp[it]+'/ocean/ocean.nc',chunks={'st_ocean': None})
        for it2 in range(size(var3docean)):
            ice_volume = (dsx[var3docean[it2]].sel(st_ocean=slice(0,z_upper))*zarea)
            iv_nh = ice_volume.sum(dim='yt_ocean').sum(dim='xt_ocean')/zarea.sum(dim='yt_ocean').sum('xt_ocean')
            ts_bgc[it2,i_yr:i_yr+size(iv_nh,0),:] = iv_nh
        i_yr += size(iv_nh,0)
        save('ts_bgc',ts_bgc)
        print(str(it)+'/'+str(exp1-exp0)+' done.')
    ts_bgc[0,:,:] = ts_bgc[0,:,:] - 273.15
#plotting time series
else:
    ts_bgc = load('ts_bgc.npy')
    figure(figsize=(15,8))
    for it in range(size(ts_bgc,0)):
        subplot(3,4,it+1)
        pcolormesh(arange(n_yrs),st_ocean,ts_bgc[it,:,:].T,rasterized=True)
        gca().invert_yaxis()
        colorbar(orientation='horizontal',label=var3docean[it]+' ('+uni3docean[it]+')')        
        xlabel('Year')
        ylabel('Depth (m)')
    tight_layout()
    savefig('ts_bgc.pdf',dpi=300)

0/63 done.
1/63 done.
2/63 done.
3/63 done.
4/63 done.
5/63 done.
6/63 done.
7/63 done.
8/63 done.
9/63 done.
10/63 done.
11/63 done.
12/63 done.
13/63 done.
14/63 done.
15/63 done.
16/63 done.
17/63 done.
18/63 done.
19/63 done.
20/63 done.
21/63 done.
22/63 done.
23/63 done.
24/63 done.
25/63 done.
26/63 done.
27/63 done.
28/63 done.
29/63 done.
30/63 done.
31/63 done.
32/63 done.
33/63 done.
34/63 done.
35/63 done.
36/63 done.
37/63 done.
38/63 done.
39/63 done.
40/63 done.
41/63 done.
42/63 done.
43/63 done.
44/63 done.
45/63 done.
46/63 done.
47/63 done.
48/63 done.
49/63 done.
50/63 done.
51/63 done.
52/63 done.
53/63 done.
54/63 done.
55/63 done.
56/63 done.
57/63 done.
58/63 done.
59/63 done.
60/63 done.
61/63 done.
62/63 done.
63/63 done.


### average over upper 100 m to assess the stability more precisely.

In [ ]:
#calculating time series
if 1 == 1:
    ts_bgc = zeros((size(var3docean),n_yrs))
    i_yr = 0    
    for it in arange(0,exp1-exp0+1):
        dsx = xr.open_dataset(datadir+l_exp[it]+'/ocean/ocean.nc')
        for it2 in range(size(var3docean)):
            ice_volume = (dsx[var3docean[it2]].sel(st_ocean=slice(0,100))*zarea.sel(st_ocean=slice(0,100)))
            iv_nh = sum(ice_volume,axis=(1,2,3))/sum(zarea.sel(st_ocean=slice(0,100)))
            ts_bgc[it2,i_yr:i_yr+size(iv_nh)] = iv_nh
        i_yr += size(iv_nh)
        save('ts_bgc_upper100',ts_bgc)
        print(str(it)+'/'+str(exp1-exp0)+' done.')
    ts_bgc[0,:] = ts_bgc[0,:] - 273.15
#plotting time series
else:
    ts_bgc = load('ts_bgc_upper100.npy')
    figure(figsize=(15,8))
    for it in range(size(ts_bgc,0)):
        subplot(3,4,it+1)
        plot(arange(n_yrs),ts_bgc[it,:])
        xlabel('Year')
        ylabel(var3docean[it]+' ('+uni3docean[it]+')')
    tight_layout()
    savefig('ts_bgc_upper100.pdf',dpi=300)

0/63 done.
1/63 done.
2/63 done.
3/63 done.
4/63 done.
5/63 done.
6/63 done.
7/63 done.
8/63 done.
9/63 done.
10/63 done.
11/63 done.
12/63 done.


### time series of globally averaged ocean fields.

In [ ]:
#calculating time series
if 1 == 1:
    ts_1docean = zeros((size(var1docean),n_yrs))
    i_yr = 0
    for it in arange(0,exp1-exp0+1):
        dsx = xr.open_dataset(datadir+l_exp[it]+'/ocean/ocean_scalar.nc')
        for it2 in range(size(var1docean)):
            ave_1y = squeeze(dsx[var1docean[it2]].groupby('time.year').mean('time'))
            ts_1docean[it2,i_yr:i_yr+size(ave_1y)] = ave_1y
        i_yr += size(ave_1y)
        print(str(it)+'/'+str(exp1-exp0)+' done.')
    save('ts_1docean',ts_1docean)
#plotting time series
else:
    ts_1docean = load('ts_1docean.npy')
    figure(figsize=(12,8))
    for it in range(size(ts_1docean,0)):
        subplot(3,2,it+2)
        title(var1docean[it])
        plot(arange(n_yrs),ts_1docean[it,:])
        xlabel('Year')
        ylabel(uni1docean[it])
    tight_layout()
    savefig('ts_1docean.pdf',dpi=300)

In [ ]:
dsx.salt_global_ave

In [ ]:
# stop the PBS job.
! pangeo.end.sh